# Neural Robot Dynamics Training on Colab

This notebook demonstrates how to setup the environment, generate a dataset, and train the NeRD model.

In [ ]:
# 1. Setup Environment
!git clone https://github.com/dhruv0000/neural-robot-dynamics.git
%cd neural-robot-dynamics
!pip install -r requirements.txt
!pip install warp-lang==1.8.0
!pip install rl_games

In [ ]:
# 2. Generate Dataset
# We generate a smaller dataset for demonstration purposes.

%cd generate

# Generate Training Data
!python generate_dataset_contact_free.py --env-name Cartpole --num-transitions 10000 --dataset-dir ../data/datasets/ --dataset-name trajectory_len-100_train.hdf5 --trajectory-length 100 --num-envs 64 --seed 0

# Generate Validation Data
!python generate_dataset_contact_free.py --env-name Cartpole --num-transitions 2000 --dataset-dir ../data/datasets/ --dataset-name trajectory_len-100_valid.hdf5 --trajectory-length 100 --num-envs 64 --seed 10

%cd ..

In [ ]:
# 3. Train Baseline Model (Transformer)
%cd train

import yaml
import os

# Load default config
with open('cfg/Cartpole/transformer.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Override dataset paths to point to the generated data
cfg['algorithm']['dataset']['train_dataset_path'] = '../data/datasets/Cartpole/trajectory_len-100_train.hdf5'
cfg['algorithm']['dataset']['valid_datasets']['exp_trajectory'] = '../data/datasets/Cartpole/trajectory_len-100_valid.hdf5'

# Reduce training parameters for quick demonstration
cfg['algorithm']['num_epochs'] = 5
cfg['algorithm']['num_iters_per_epoch'] = 100

# Save the modified config
with open('colab_config.yaml', 'w') as f:
    yaml.dump(cfg, f)

# Run training
!python train.py --cfg colab_config.yaml --logdir ../data/logs/baseline

In [ ]:
# 4. Train Mamba Model
# We use the same config but add the --novelty mamba flag
!python train.py --cfg colab_config.yaml --novelty mamba --logdir ../data/logs/mamba

In [ ]:
# 5. Train Unroll Model
# We use the same config but add the --novelty unroll flag
!python train.py --cfg colab_config.yaml --novelty unroll --logdir ../data/logs/unroll

In [ ]:
# 6. RL Evaluation
# Evaluate the trained models using the pretrained RL policy

# Baseline
print("Evaluating Baseline Model...")
!cd ../eval/eval_rl && python run_rl.py --rl-cfg ./cfg/Cartpole/cartpole.yaml --playback ../../pretrained_models/RL_policies/Cartpole/0/nn/CartpolePPO.pth --num-envs 1 --num-games 5 --env-mode neural --nerd-model-path ../../data/logs/baseline/nn/best_model.pt

# Mamba
print("Evaluating Mamba Model...")
!cd ../eval/eval_rl && python run_rl.py --rl-cfg ./cfg/Cartpole/cartpole.yaml --playback ../../pretrained_models/RL_policies/Cartpole/0/nn/CartpolePPO.pth --num-envs 1 --num-games 5 --env-mode neural --nerd-model-path ../../data/logs/mamba/nn/best_model.pt

# Unroll
print("Evaluating Unroll Model...")
!cd ../eval/eval_rl && python run_rl.py --rl-cfg ./cfg/Cartpole/cartpole.yaml --playback ../../pretrained_models/RL_policies/Cartpole/0/nn/CartpolePPO.pth --num-envs 1 --num-games 5 --env-mode neural --nerd-model-path ../../data/logs/unroll/nn/best_model.pt